In [5]:
import pandas as pd
import numpy as np
import itertools

# display options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# custom helper functions
%load_ext autoreload
%autoreload 2
from helper.general_helper import *

# keras
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from keras.models import Model

# sklearn
from sklearn.model_selection import train_test_split

# view plotly in jupyter 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
df = pd.read_csv('../../merged_df.csv')

In [6]:
# create train and test data
df = df[['reviewerID', 'asin', 'overall']]

train, test = train_test_split(df, test_size=0.2, random_state=42)

##### Embedding

In [ ]:
n_asins = len(df.asin.unique())
n_reviewers = len(df.reviewerID.unique())



# creating asin embedding path
asin_input = Input(shape=[1], name="asin-input")
asin_embedding = Embedding(n_asins+1, 5, name="asink-embedding")(asin_input)
asin_vec = Flatten(name="flatten-asin")(asin_embedding)

# creating user embedding path
reviewer_input = Input(shape=[1], name="reviewer-input")
reviewer_embedding = Embedding(n_reviewers+1, 5, name="reviewer-embedding")(reviewer_input)
reviewer_vec = Flatten(name="flatten-reviewer")(reviewer_embedding)

##### Model Creation

In [ ]:
# model building
conc = Concatenate()([asin_vec, reviewer_vec])

layer_1 = Dense(128, activation='relu')(conc)
layer_2 = Dense(32, activation='relu')(layer_1)
output = Dense(1)(layer_2)

model = Model([reviewer_input, asin_input], output)
model.compile('adam', loss='mae', metrics=['rmse'])

In [ ]:
model.save('../../basic_neural_net.h5')

# load model:
# model = load_model(../../basic_neural_net.h5)

##### Model Evaluation

In [ ]:
model.evaluate([test.reviewerID, test.asin])

##### Make Predictions

In [ ]:
# recomendations for first user
asin_data = np.array(list(set(df.asin)))
book_data[:5]

In [ ]:
reviewer = np.array([1 for i in range(len(asin_data))])
reviewer[:5]

In [ ]:
preds = model.predict([reviewer, asin_data])
preds = np.array([a[0] for a in preds])
recommended_asin = (-preds).argsort()[:5]
recommended_asin